In [79]:
import spacy
import nltk
from nltk import Tree
import re

In [75]:
nlp = spacy.load("en_core_web_sm")

In [374]:
def split_sent(doc):
    temp = []
    nouns = [tok.text for tok in doc if tok.tag_.startswith('NN')]
    for tok in doc:
        temp.append(tok.text)
        if tok.dep_=='cc':
            left_right_tokens = [str(doc[tok.i-1]), str(doc[tok.i+1])]
            if left_right_tokens[0] not in nouns or left_right_tokens[1] not in nouns:
                yield temp
                temp = []
    if temp:
        yield temp

In [345]:
def redefine_group(lst):
    temp = ''
    for sent in lst:
        doc = nlp(sent)
        dep_str = ''.join([tok.dep_ for tok in doc])
        pos_str = ''.join([tok.pos_ for tok in doc])
        
        if 'subj' in dep_str or 'VERB' in pos_str or 'AUX' in pos_str:
            if temp:
                yield temp.strip()
                temp = ''
            temp+=' '+sent
        else:
            temp+=' '+sent
    if temp:
        yield temp.strip()

In [91]:
def get_tree(sent):
    grammar = ('''
            n_chunk: {<DT>? <JJ>* <NN.*>+ (<IN>? <NN.*>+)?}
            ''')
    chunkParser = nltk.RegexpParser(grammar)
    tagged = nltk.pos_tag(nltk.word_tokenize(sent))

    tree = chunkParser.parse(tagged)
    return tree

def tree2dict(tree):
    return {tree.label(): [tree2dict(t) 
            if isinstance(t, Tree) else t for t in tree]}

def get_n_chunks(lst):
    tree = get_tree([(i[0], i[2]) for i in lst])
    chunks = tree2dict(tree)
    #print(len(re.findall('n_chunk',str(chunks))),'\n\n')
    
    for i in chunks['S']:
        if type(i) is dict and 'n_chunk' in i.keys():
            yield (i['n_chunk'])

In [346]:
def get_subject(text):
    prv_tok_dep = ""    # dependency tag of previous token in the sentence
    prv_tok_text = ""   # previous token in the sentence
    prefix = ""
    modifier = ""
    sub = ""
    subs = []
    for tok in nlp(text):
        # if token is a punctuation mark then move on to the next token
        if tok.dep_ != "punct":
            # check: token is a compound word or not
            if tok.dep_ == "compound":
                # if the previous word was also a 'compound' then add the current word to it
                if prv_tok_dep == "compound":
                    prefix = prefix+" "+tok.text
                else:
                    prefix = tok.text

            # check: token is a modifier or not
            if tok.dep_.endswith("mod") == True:
                modifier = tok.text
                # if the previous word was also a 'compound' then add the current word to it
                if prv_tok_dep == "compound":
                    modifier = prv_tok_text +" "+ tok.text

            if tok.dep_.find("subj") == True:
                sub = modifier +" "+ prefix + " "+ tok.text
                prefix = ""
                modifier = ""
                prv_tok_dep = ""
                prv_tok_text = ""
        prv_tok_dep = tok.dep_
        prv_tok_text = tok.text
#        if sub:
#            subs.append(sub.strip())
    return sub

In [108]:
'''def add_subject(lst):
    temp = []
    for i in lst:
        #print(list(get_n_chunks(i)))
        if not any(j[-1][:5]=='nsubj' for j in i):
            temp.append(get_subject(i))
        else:
            print(i)'''

In [405]:
def process_text(text):
    doc = nlp(text)
    group_sent = [' '.join(i) for i in list(split_sent(doc))]
    sorted_sents = list(redefine_group(group_sent))
    clean_sents = []
    for sent in sorted_sents:
        deps = [tok.dep_ for tok in nlp(sent)]
        if deps[-1]=='cc':
            sent = sent[:-len(sent.split()[-1])]
            clean_sents.append(sent.strip())
        else:
            clean_sents.append(sent)
    new_sents = []
    prev_sent = ''
    for sent in clean_sents:
        if not any('subj' in mystr for mystr in [tok.dep_ for tok in doc]):
            sent = get_subject(prev_sent)+" "+sent
        prev_sent = sent
        new_sents.append(sent.strip())
    return new_sents

In [406]:
text = "Two passengers with a travel history to Italy and South Korea have been shifted to the quarantine and the ward at a government medical college hospital in Jammu, officials said on Wednesday."
#text = "Jammu and Kashmir has been put on alert to deal with any threat emerging and it is due to coronavirus and even though no positive case has been found in the Union Territory so far and so fine"
process_text(text)

['Two passengers with a travel history to Italy and South Korea have been shifted to the quarantine',
 'the ward at a government medical college hospital in Jammu , officials said on Wednesday .']

In [429]:
hindu = """Navneet Kaur Rana, MP from Amravati Lok Sabha constituency, Maharashtra, wears a protective mask as she arrives in Parliament on Wednesday, March 5, 2020. India reports 29 cases so far. Fifteen Italian tourists, including the wife of an infected Italian, and eight Indians have tested positive for on March 4, taking the number of cases to 29 in the country. Two cases with a high viral load were A Paytm employee tested positive in Gurgaon. The company said in a statement that the person had travel history to Italy. The Health Ministry also confirmed the case which came to light in the evening.The Union Health Ministry has set up control room to tackle coronavirus-related queries. If you have fever, cough and difficulty in breathing, call your State helpline number or Ministry of Health & Family Welfare, Government of Indias 24X7 control room number +91-11-23978046, a press release said. For any health related query people can email at ncov2019@gmail.com. Here are the updates:.NEW DELHI 8.35 P.M.Paytm employee in Gurgaon tests positive for COVID-19. A Paytm employee in Gurgaon has tested positive for novel coronavirus, the company said in a statement on Wednesday. It said the employee had recently returned from a vacation in Italy, one of the worst-hit countries from coronavirus. Paytm has advised all its employees to work from home for the next couple of days while the Gurgaon unit gets sanitised, according to the statement. NEW DELHI 8.05 P.M.CBSE to allow students to carry masks, hand sanitiser during boards exams. Students appearing for class 10 and 12 board examination will be allowed to carry masks and hand sanitiser in the exam centre in view of coronavirus scare, the Central Board of Secondary Education (CBSE) announced on Wednesday. Face masks and sanitisers may be carried by students in the examination centre, CBSE Secretary Anurag Tripathi said. Class 10 and 12 board examinations began on February 15. WASHINGTON 8.00 P.M.IMF members pledge all available resources against COVID-19. The 189 members of the International Monetary Fund pledged on Wednesday to bring all available resources to bear to help countries combat the impact of the new coronavirus epidemic. Following an unusual conference call of the IMFs governing body, the members said they were united in addressing the challenges posed by COVID-19. We have called upon the IMF to use all its available financing instruments to help member countries in need, the statement said.We are determined to provide the necessary support to mitigate the impact, especially on the most vulnerable people and countries. Also read. PUNJAB 7.50 P.M.70,000 people screened for COVID-19 in Punjab, all negative: Health Minister. As many as 70,000 people have been screened for the deadly coronavirus in Punjab so far, Health Minister Balbir Singh Sidhu informed the state assembly on Wednesday. Their test reports have been found negative, he said during the Zero Hour. The Health Minister further informed the House that medical teams have been deputed at the Amritsar and Mohali airports for screening of passengers.Besides, medical teams have also been deputed at the Attari-Wagah border and the Dera Baba Nanak in Gurdaspur. NEW DELHI 7.20 P.M.Rashtrapati Bhavan not to hold traditional Holi gatherings. ''With alertness and safeguards, we all can help contain the outbreak of COVID-19 Novel Coronavirus. In a precautionary measure, the Rashtrapati Bhavan will not hold the traditional Holi gatherings,'' tweeted the Rashtrapati Bhavan. ITALY 7.15 P.M.Italy to shut down schools across the country. Italy is closing all schools and universities from Thursday until mid-March to help curb the spread of the novel coronavirus, Ansa and other Italian news agencies cited government sources as saying.The measure was one of several being considered by the government at a meeting on Wednesday to combat COVID-19, which has killed 79 people in Italy and infected more than 2,500, the most in Europe.HYDERABAD 6.50 P.M.Four family members of Hyderabad's first COVID-19 patient test negative.Four family members of the first COVID-19 positive patient from Hyderabad tested negative for COVID-19.In another case, two more samples were sent to Pune for clarity."Two more samples were sent to the National Institute of Virology Pune to get clarity. Results would be known by Thursday evening," said Telangana Health Minister Eatala Rajender.The Minister added that private hospitals now which have capacity are allowed to provide treatment for COVID-19 from today. Earlier, isolation, treatment for COVID was allowed only Gandhi Hospital, Hyderabad. Now, private hospitals can admit, treat, and send samples to Gandhi Hospital for tests.NEW DELHI 6.40 P.M.HRD Ministry advisory to States, CBSE.Union HRD Ministry directs Chief Secretaries of States and CBSE to spread awareness among students on basic protective measures against coronavirus.SAUDI ARABIA 6.30 P.M.Saudis suspend Umrah pilgrimage to Mecca.Saudi Arabia has suspended the year-round umrah pilgrimage to the holy cities of Mecca and Medina over coronavirus fears, the Interior Ministry said. (Umrah is the year-round pilgrimage taken to Mecca, while Hajj is taken during the prescribed period in a year.).Saudi has decided to suspend umrah temporarily for citizens and residents in the kingdom, the Ministry said in a statement carried by the official Saudi Press Agency.The decision comes after Saudi Arabia last week suspended visas for the pilgrimage and barred citizens from the six-nation Gulf Cooperation Council from entering two of Islam's holiest cities. AFP.ETHIOPIA 6.10 P.M.African airlines face $40 million hit in 2020 from coronavirus.Coronavirus disruption could cost African airlines $40 million in revenue this year, a global industry body said on Wednesday, a potentially devastating hit to often struggling airlines counting on lucrative Chinese routes to fund expansion.The global hit to the aviation industry is projected to be $29 billion this year - a 4.7% industry-wide drop in revenue per passenger kilometre, the International Air Transport Association (IATA) has said.The blow to African airlines could be as much as $40 million, IATA's special envoy to Africa, Raphael Kuuchi, said at an aviation conference in Addis Ababa.IRAN 6.05 P.M.Iran rejects U.S. virus aid offer amid 'vicious' sanctions.Iran's President Hassan Rouhani on Wednesday dismissed a US offer to help the Islamic republic fight its coronavirus outbreak, charging that vicious American sanctions are depriving the country of medicine.Iran has scrambled to halt the rapid spread of the virus that has claimed 92 lives out of 2,922 confirmed infections in the past two weeks.Those who have deprived the people of even medicine and food through sanctions, who have done the most vicious things.. they appear with a mask of sympathy and say that we want to help the nation of Iran, Mr. Rouhani said, in a clear reference to the United States.U.S. President Donald Trump had said Saturday he was ready to aid Iran with the virus outbreak if the Islamic republic asked for assistance.JAMMU AND KASHMIR 6.00 P.M.Two passengers with travel history to Italy, South Korea quarantined at Jammu hospital.Two passengers with a travel history to Italy and South Korea have been shifted to the quarantine ward at a government medical college hospital in Jammu, officials said on Wednesday.Jammu and Kashmir has been put on alert to deal with any threat emerging due to coronavirus, even though no positive case has been found in the Union Territory so far, officials said.The tests of the two persons have been sent to Delhi and symptomatic passengers have been instructed for home quarantine, they said.WEST BENGAL 4.40 PM.COVID-19 being hyped to divert attention from Delhi riots: Mamata Banerjee.West Bengal Chief Minister Mamata Banerjee on Wednesday alleged that a section of the media was playing up the spread of Coronavirus (COVID-19) to deflect public attention from the Delhi riots.Addressing a rally at Buniyaadpur in Dakshin Dinajpur district of north Bengal the Trinamool Congress chairperson accused the media of playing up the outbreak of COVID-19.4.10 PM.A number of international sports events have been cancelled, postponed or shifted to other cities due to the new virus outbreak.KARNATAKA 3.50 PM.Karnataka appeals to shun coronavirus rumours.Karnataka Health and Family Welfare Minister B. Sriramulu on Wednesday appealed to consider only official information and shun rumours on Coronavirus.I request that you consider only official information, without heeding any rumours that are spreading across various social networking sites, tweeted Sriramulu.He said the state Health and Family Welfare department has taken all possible measures to prevent the spread of infection.So far, 40,207 passengers have been inspected at the Bengaluru airport. Blood tests were performed on 251 people and 238 were negative. The report of the remaining 13 people is yet to come, he said.Explaining the 24-year-old city-based techies case, who tested positive for Coronavirus in Hyderabad, Mr. Sriramulu said his residence and office in Bengaluru were cleaned on Tuesday from the infection.And 25 of his office colleagues were contacted, and one was hospitalized in a preoperative manner, and a blood sample was sent for testing, he added.DELHI 3.45 PM.17 Indians abroad infected with coronavirus.Seventeen Indians have been infected with coronavirus abroad, the Ministry of External Affairs (MEA) said on Wednesday.Responding to a question in Lok Sabha, Minister of State for External Affairs V Muraleedharan said 16 cases were reported from Japan on the cruise ship and one from the UAE.He further said that the government evacuated a total of 766 persons -- 723 Indian nationals and 43 foreigners -- from China.Similarly, 119 Indians have been brought back to India from the cruise ship in Japan, Mr. Muraleedharan said.RAJASTHAN 3.00 PM.'215 people came in contact with COVID-19 affected Italian tourists'.Rajasthan Health Minister Raghu Sharma said at least 215 people came in contact with the COVID-19-affected group of Italian tourists.Mr. Sharma informed the Assembly that swab samples of 93 people have been taken. A total of 51 suspected patients have been tested negative whereas reports of 41 others are still pending.As many as 53 people in Jhunjhunu, 14 each in Jodhpur and Jaisalmer, 44 in Bikaner, six in Udaipur and 76 people in Jaipur came in contact with the group. The 15 Italian tourists who were tested positive for COVID-19 had reached Delhi, via Jaipur and Agra. The tourists entered Rajasthan via road from Delhi.He said that though there is no prescribed medication or vaccine to treat the disease, the suspected patients in the state are being given swine flu medication as a precautionary measure.Purchase orders have been given for 50,000 personal protective equipment kits and 5 lakh each of N95 and triple layer masks to deal with the problem, the minister said.2.35 PM.Helpline numbers.The Union Health Ministry has set up control room to tackle coronavirus-related queries.If you have fever, cough and difficulty in breathing, call your State helpline number or Ministry of Health & Family Welfare, Government of Indias 24*7 control room number +91-11-23978046, a press release said.For any health related query people can email at ncov2019@gmail.com.Ministry of Health #StayHome #StaySafe.@MoHFW_INDIA. Some simple hygienic practices can help in protecting against . Please share this information for wider awareness. Please also note our 24*7 Control Room Helpline number.549.442 people are talking about this.DELHI 2.20 PM.Let's fight coronavirus together: Kejriwal.As fresh cases are being reported, people in the national capital are scrambling to buy face masks.Delhi Chief Minister Arvind Kejriwal said the government has enough N95 masks in stock and will check if such masks are sold at higher price in black market.N95 masks are recommended for blocking viruses.Mr. Kejriwal also asked Delhiites to avoid public spaces and wash hands often. The public needs to be aware and come together to fight coronavirus like Delhi fought Dengue last year, he said.Mr. Kejriwal announced that he will not be celebrating Holi on account of the the violence in Delhi and also due to the virus threat. - Jaideep Deo Bhanj.DELHI 1.30 PM.Trying to get Indians back from Iran: Harsh Vardhan.Union Health Minister Harsh Vardhan has said that all international passengers will be screened for coronavirus symptoms."We are trying to get Indians back from Iran. We are working on strategy," Dr. Vardhan said.If the Iranian government lent its support, a testing facility could be set up there as well. "This will help bring back Indian citizens from Iran after a thorough screening," he said.An estimated 1,200 Indians, mostly students and pilgrims, are in Iran currently. Iran has so far reported over 2,000 coronavirus cases and 77 deaths.DELHI 1 P.M.India reports 28 cases so far.Fourteen Italian tourists and one Indian driver have tested positive for COVID-19. They were part of the same group as the Italian man who was tested positive for COVID-19 in Rajasthan.Six family members of Delhi man who was found to have contracted COVID-19, have been tested positive. They hail from Agra.All the Italian tourists were at Chhawala ITBP medical camp currently. The Principal Secretary to PMO will also be holding a review meeting on Wednesday to oversee the COVID-19 preparedness.The official confirmed cases in India as of today stands at 28. It includes 17 from Jaipur, 6 in Agra, 3 in Kerala and one each in Telangana and Delhi. Three patients from Kerala have recovered from the illness.ODISHA.Odisha govt advisory to keep coronavirus at bay.The Odisha government has in its list of dos and donts asked people to avoid hugging, shaking hands while greeting and keep their workplace clean to ward off coronavirus infection.The state government issued the workplace advisory based on the containment plan prepared by the Health Ministry to tackle the COVID-19 disease.Asking the employers to ensure workplace cleanliness, it mentioned, Surfaces (desks and tables) and objects (telephone, keyboards) need to be regularly wiped with disinfectant containing 70 per cent alcohol.Soaps or sanitizing hand rub should be provided near wash basins and water supply should be ensured as frequent hand washing kills the virus and prevents spread of Covid-19, it said.Posters should be displayed promoting respiratory hygiene, and tissue papers or masks should be made available to the employees.The staff should be advised home isolation whenever they develop flu like symptoms, it said.TELANGANA.Two samples from Hyderabad sent to NIV Pune for further tests.Out of 47 samples collected from people in Telangana, viral load was found to be high in two cases. Their samples are sent to National Institute of Virology, Pune for further testing. 45 have tested negative.Among the two, whose samples have been sent for further tests, one was a hospital staff treating the COVID-19 patient. The other person is said to have travelled to Italy recently. .Modi not to take part in Holi milan.Prime Minister Narendra Modi on Wednesday said he has decided not to participate in any Holi milan programme as experts have advised to reduce mass gatherings to avoid the spread of COVID-19.Experts across the world have advised to reduce mass gatherings to avoid the spread of COVID19 Novel Coronavirus. Hence this year, I have decided not to participate in any Holi milan programme, the PM tweeted.This year, Holi is on March 10.Watch COVID-19: Dos and don'ts from the Health Ministry.Here's a video on how we can protect ourselves and others, a list of dos and dont's released by the Health Ministry.COVID-19: Do's and don'ts from the Health Ministry.UTTAR PRADESH.109 of 137 samples test negative in Uttar Pradesh.Of the 137 samples sent for testing for in Uttar Pradesh, 109 have found to be negative while six samples that tested positive have been sent for confirmation, the government said. The results of 22 cases were still awaited, said the State Control Room, Integrated Disease Surveillance Programme Vector Borne Disease Division Directorate of Health Services.VISAKHAPATNAM.Malaysia returned father-daughter duo admitted to hospital in Vizag on coronavirus suspicion.Officials from the Medical and Health Department hospitalised a person and his daughter, who came from Malaysia, for observation, after the duo were found having coronavirus symptoms.KAKINADA.Techie with suspected COVID-19 put in Isolation Ward in Kakinada Govt. Hospital in A.P.A 35-year-old Hyderabad-based Software Engineer suspected to have been infected with coronavirus has been admitted to the Isolation Ward at the Government General Hospital (GGH) here in East Godavari district in the early hours of Wednesday.BENGALURU.One Intel employee in Bengaluru potentially exposed to coronavirus, under quarantine.Tech giant Intel has said one of its employees in Bengaluru has potentially been exposed to and is currently under quarantine. The company also said it has implemented precautionary measures like travel restrictions, increased frequency of office sanitisation, and work-from-home provisions in India.HYDERABAD.Hyderabad COVID-19 patient questioned on contacts.In his 24 years, perhaps Monday was the first time that this Hyderabad man was persistently asked by someone to give minute-by-minute details of who all he met, places he had been to, how much time he spent with them from the moment he stepped into Telangana.CHINA.China reports fall in cases, South Korea death toll hits 32.China on March 4 reported 38 more deaths from the but a fall in fresh cases for a third consecutive day. The death toll nationwide is now 2,981, the National Health Commission said, with more than 80,200 people infected in total.COVID-19 has higher death rate than influenza: WHO chief.Contrary to the general notion that has lower death rate compared with influenza (commonly called as seasonal flu), COVID-19 has globally caused more than three times the number of deaths in reported cases than influenza."""
nyt = """The workers conditions were not immediately available, but public health officials in Alameda County and Solano County said in a news release that the workers were isolated in their homes.The news came after new cases were reported in both Washington State and Rhode Island. Alex M. Azar II, the health and human services secretary, said on CBSs Face the Nation on Sunday that the government planned a radical expansion in testing capacity.Globally, the number of infections has risen to more than 88,000. reported an additional 202 infections and 42 deaths from the virus, bringing its total number of confirmed infections to more than 80,000 and total of deaths from the virus to exceed 3,000.[Read: .].In Washington, two new patients in King County, both men in their 60s, were in critical condition, officials said. The state became the site of the countrys first coronavirus death on Saturday.Updated 1m ago.More live coverage: . over the coronavirus, and researchers who studied two cases there say that the virus may have been , with the possibility that up to 1,500 people may have been infected.MORE ON THE RESEARCH.our full story on the .Health officials in Rhode Island announced Sunday that a second person in that state tested positive for coronavirus after returning from a school trip to Europe.That person, a teenager, had minor symptoms. Officials had earlier announce that an adult who went on the trip tested positive. Eighty-eight cases of coronavirus had been identified in the United States as of Sunday night, including 23 announced over the weekend.[Do you know anyone who lives or works at Life Care Center in Kirkland, Wash.? If so, please email our reporter, Mike Baker, at .].Which is worse, coronavirus or flu?.A doctor with patients on Feb. 21 at a gym that was turned into a temporary hospital in Wuhan, China.Credit.Chinatopix, via Associated Press.The coronavirus and the flu are often compared these days. But what are their ?.So far, the coronavirus seems to be deadlier. On average, the seasonal flu strain kills about 0.1 percent of people who become infected. Early estimates of the death rate in the coronavirus outbreaks epicenter in Wuhan, China, have been around 2 percent.The rate could fall if it turns out that many cases arent detected because they are so mild or even symptom-free.As with influenza, the coronavirus is most dangerous to people over the age of 65, or who have chronic illness or a weak immune system.[Read: .].And so far, the flu has sickened more people than the coronavirus. In the United States, there have been 32 million cases of flu, several hundred thousands hospitalizations and 18,000 deaths, according to the C.D.C. By contrast, about 88 people in the United States have been infected with the new coronavirus, and there have been two deaths.One area where the two ailments diverge is treatment. There is no approved antiviral drug for the coronavirus, but several are being tested. For those infected with any viral illness, doctors recommend rest, medicine to reduce pain and fever, and fluids to avoid dehydration. For the flu, doctors can offer and they tend to work best within a day or two of when symptoms start.There are no coronavirus vaccinations available, but one may be available in a year or two. Flu vaccines are widely available and generally 40 percent to 60 percent effective.CORONAVIRUS VERSUS INFLUENZA. comparing the two diseases. We also looked at the question of .Amazon says 2 employees infected, as tech companies introduce new restrictions worldwide.Two Amazon employees in Europe have contracted the coronavirus, the company said, and other tech firms have begun taking more drastic measures to prevent their employees around the world from being affected by the outbreak.On Sunday, Amazon spokesman Drew Herdener said that the internet retailer was supporting the affected employees, who were in Milan and are now in quarantine. He added that Amazon, which is based in Seattle, did not know of any employees in the United States who had become sick.Late last week, Amazon, the second largest private employer in the United States, indefinitely halted all nonessential travel, including trips within the U.S. Any employee looking to book critical travel must get approval from a vice president, and recruiters were told that many hiring interviews were to be done by video conference. The company also employees to refrain from scheduling meetings that required flights until at least the end of April.Sign up to receive our daily Coronavirus Briefing, an informed guide with the latest developments and expert advice.Sign Up.Other tech firms are also working to secure their offices while minimizing the potential for the spread of the virus. Facebook has advised employees to reach out if they feel ill and to feel free to work from home, according to two company employees familiar with the companys plans. Over the weekend, the company also said in an internal memo that it would no longer allow social visits from non-employees at any of its global offices.And on Sunday, that it would also restrict all nonessential business travel for its employees and partners.Scotland and the Dominican Republic report their first cases, and Ecuador reports more.The coronavirus continued to spread around the world on Sunday.Ecuadors health ministry said it had six confirmed cases of coronavirus, with one person in critical condition. All are part of the same family, the authorities said. The five noncritical patients were in home isolation as a protective measure and under observation, Catalina Andramuno Zeballos, the countrys health minister, said in a news conference.Scotland said one person had tested positive people tested.We have to be prepared for the possibility of a significant outbreak of coronavirus in the weeks to come, Nicola Sturgeon, Scotlands first minister, with Sky News.In the Dominican Republic, a 62-year-old man a tourist visiting the country from Italy tested positive and was in stable condition, the countrys health minister, Rafael Sanchez, said at a news conference. Another tourist, a 56-year-old man from France, is under observation at a hospital and awaiting test results for coronavirus.Louvre closes for coronavirus meeting as cases rise in Europe.The Louvre closed on Sunday for a coronavirus meeting.Credit.Yoan Valat/EPA, via Shutterstock.Long lines of visitors were locked out of the Louvre in Paris on Sunday after the museum, the worlds most visited, said it was holding a staff meeting about the coronavirus outbreak. that the museum cannot open at the moment because of an informational meeting on the public health situation linked to coronavirus prevention measures.A spokeswoman told reporters that the museum, in central Paris, was expected to reopen after the meeting among management, employees and the staff doctor, but it later said on Twitter that it would remain closed.The Louvre Museum cannot open today, Sunday March 1st. We invite you to write to Assistance-billetterie@louvre.fr for a refund for any entrance tickets for today. We apologize for any inconvenience and will keep you informed of developments. Musee du Louvre (@MuseeLouvre) .The move came as the number of confirmed coronavirus cases continued to rise in Europe.Italy, the center of the outbreak on the continent, has a total of 1,694 confirmed cases and 34 deaths. France has reported 100 cases and two deaths. And Germany said on Sunday that cases there had risen to 117, including 66 in North Rhine-Westphalia, its most populous state.Ireland reported its first case, the countrys Health Protection Surveillance Center , while in neighboring Britain officials said that the number of cases there had risen to 35. A school in Reading, in southeastern England, that a staff member had tested positive, forcing the school to shut for some days to allow for a deep clean, the schools head teacher said in a message to the pupils parents.In Norway, two more staff members at Oslo University Hospital tested positive for the virus, a hospital spokesman said on Sunday, raising the total number of cases in the country to 17, while Sweden has .Flights are suspended amid an outbreak in Italy.The Milan Cathedral remained closed to the general public on Sunday.Credit.Andrea Mantovani for The New York Times.The uptick in coronavirus cases in Italy has already prompted new local safety measures and mass tourist cancellations, .This weekend, the impact of the outbreak stretched even further.Delta Air Lines said flights between New York and Milan, becoming yet another travel business to alter its operations in light of the outbreak in Italy. On Saturday it would suspend flights to Milan from New York and Miami, citing a reduction in demand.The airlines suspensions came as President Trump on Saturday announced elevated restrictions on foreign travel to the parts of Italy most affected by the virus.Delta said it planned to resume flights between Milan and New York in early May, and American said its flights would resume on April 25. The airlines said travelers affected by the suspensions could rebook their flights after service began again, or could get a refund.RADICAL EXPANSION OF TESTING. The C.D.C. said it would , while White House officials said they were distributing tens of thousands of testing kits.Seoul seeks a murder inquiry into leaders of a church linked to the outbreak.Government officials temporarily closing down a branch of the Shincheonji Church of Jesus last week.Credit.Ryu Hyung-Geun/Yonhap, via Associated Press.Officials from the city of Seoul, South Korea, on Sunday asked prosecutors to investigate and other top leaders of the sect on murder and other criminal charges.[Read: .].Officials believe that the church has contributed to the countrys rising death toll from the virus which reached 18 as of Sunday by failing to provide disease-control officials with an accurate list of church members and by interfering with the governments efforts to fight the outbreak.In a Facebook post, Mayor Park Won-soon of Seoul said the churchs behavior was tantamount to murder through to willful negligence.Officials say that nearly 60 percent of the 4,212 confirmed cases in South Korea are in members of in the southeastern city of Daegu or people who came into contact with them.Prosecutors have yet to decide whether to begin a formal investigation into the founder, Lee Man-hee, and other sect leaders.Shincheonji officials did not immediately respond to a request for comment on Sunday. The church has said it was fully cooperating with the government, calling itself the victim of a witch hunt.South Korean officials also said on Sunday that some church members had visited the Chinese city of Wuhan in January. The global outbreak is believed to have begun in a in Wuhan.Officials in South Korea have been trying to figure out how the virus reached the congregation. The church has acknowledged having members in Wuhan, but it said none of them had visited South Korea since December.THE WHITE HOUSE RESPONSE.and more on .Remaining Diamond Princess passengers and crew are cleared, Japan says.The Diamond Princess cruise ship docked in Yokohama, Japan.Credit.Kazuhiro Nogi/Agence France-Presse Getty Images.A Japanese official said on Sunday that all of the passengers and crew members from the Diamond Princess cruise ship who spent more than two weeks quarantined onboard had been cleared of the coronavirus, but a man who was a passenger on the ship has become the first in Australia to die of the virus.The cruise ship, on which hundreds of Americans were passengers, represents the largest concentration of coronavirus cases outside China, with several hundred infections.Japans health minister, Katsunobu Kato, said in a press briefing on Sunday that 238 crew members would be quarantined for 14 days at a government facility in Saitama Prefecture outside Tokyo.The ministry says that 149 crew members contracted the virus onboard the ship. Hundreds of other crew members have been evacuated to their home countries.The health minister also confirmed that an eighth public official who worked onboard the cruise ship during the quarantine had tested positive on Friday.Cases are up to 385 in Iran, and flights are blocked to and from Iraq.In Tehran on Saturday. Iran has been at the center of the viruss spread in the Middle East.Credit.Vahid Salemi/Associated Press.Iran, the epicenter of the coronavirus outbreak in the Middle East, said on Sunday that 385 new cases had been detected in the country this weekend, raising its total number of officially confirmed cases to 987, state news media reported.The continued rise in infections has prompted the United States to ban all travel to Iran and bar entry to foreign citizens who have visited Iran in the past 14 days.An Iranian health ministry spokesman, Kianoush Jahanpur, said on Sunday that the countrys death toll from the virus had risen to 54, the Islamic Republic News Agency reported.Iran has temporarily closed schools and universities and canceled public events like concerts in an effort to curtail the viruss spread. But secrecy and confusion over the outbreak in the country have fueled suspicions that its number of cases is far higher than officially acknowledged.In neighboring Iraq, the government grappled with how tightly to shut down daily life as some international flights were suspended.Turkish Airlines on Sunday suspended flights to and from Iraq, saying initially that the suspension was for 24 hours but later suggesting that it could last at least three or four days.Democratic candidates use the coronavirus to attack President Trump on the economy.Mike Bloomberg at a campaign stop in Blountville, Tenn., on Friday.Credit.Brittainy Newman/The New York Times.President Trumps greatest strength with voters has been his stewardship of the economy.But after the effects of the coronavirus dealt stock markets their , Democratic presidential candidates opened . Senator Elizabeth Warren of Massachusetts and former Mayor Michael R. Bloomberg of New York have revamped their stump speeches to portray themselves as the type of president the United States needs to endure a potential economic and public health crisis.Mr. Bloomberg will deliver a three-minute recorded address on the coronavirus outbreak in an ad on network television Sunday night, according to his presidential campaign.It was not immediately clear how much of Mr. Bloombergs personal fortune he spent on the elevated platform. The ad is set to air around 8:30 p.m. Eastern on CBS and NBC, and media executives estimated that it could cost the campaign $1.25 million to $3 million, depending on whether the networks charged a premium for a last-minute purchase.Before its scheduled airing, the Bloomberg campaign released the video of his remarks, pitching it as an unprecedented candidate address.The public wants to know their leader is trained, informed and respected, he says.Xi Jinping says China must do better.Xi Jinping, Chinas leader, having his temperature checked in Beijing last month.Credit.Ju Peng/Xinhua, via Agence France-Presse Getty Images.Chinas initial response to the coronavirus epidemic was marred by policy stumbles that fueled public anger, the nations leader, Xi Jinping, said in published speech excerpts that laid out his ideas for strengthening the countrys defenses against such outbreaks.Mr. Xis comments, drawn from two internal speeches that he made in February, , or Seeking Truth, the ruling Communist Partys leading journal. They seemed intended to highlight the policy and legal changes that Mr. Xi intends to push to confront the epidemic.Those include banning the trade in wildlife that scientists believe may have let the coronavirus jump from animals into the human population; more effective monitoring of potential epidemics; and stronger coordination to direct emergency medical supplies when an outbreak happens.While praising the Chinese governments response to the crisis, Mr. Xi also acknowledged problems, using blunter language than he had in previous public comments on the epidemic.Some localities and departments were at a loss in how to react to this sudden epidemic, Mr. Xi said. Some protective measures went through abrupt changes, and in some areas there was even lawless and criminal conduct that seriously impeded containing the epidemic, and there was public dissatisfaction about this.Mr. Xi did not elaborate on what he meant by criminal conduct. Chinese news media outlets have reported cases of officials neglecting stricken families, as well as crude, unhygienic efforts to transfer patients.,Outbreak Strikes Seattle Area as Testing Is Scrutinized.The epidemic continues to expand rapidly worldwide, topping 90,000 cases and 3,000 deaths. U.S. officials say virus testing kits will soon become widely available...0:00/1:30.Pence Announces Travel Screenings for Italy and South Korea.Vice President Mike Pence and Dr. Deborah L. Birx, the White Houses coronavirus response coordinator, gave updates on the governments response to the virus.It is remarkable to think that there may well be a vaccine going to clinical trials within the next six weeks."""
#sents = [sent.strip()+'.' for sent in re.split(r'\.', corpus)]
sents = [sent.strip() for sent in nltk.sent_tokenize(nyt)]
len(sents)

59

In [430]:
sents

['The workers conditions were not immediately available, but public health officials in Alameda County and Solano County said in a news release that the workers were isolated in their homes.The news came after new cases were reported in both Washington State and Rhode Island.',
 'Alex M. Azar II, the health and human services secretary, said on CBSs Face the Nation on Sunday that the government planned a radical expansion in testing capacity.Globally, the number of infections has risen to more than 88,000. reported an additional 202 infections and 42 deaths from the virus, bringing its total number of confirmed infections to more than 80,000 and total of deaths from the virus to exceed 3,000.',
 '[Read: .',
 '].In Washington, two new patients in King County, both men in their 60s, were in critical condition, officials said.',
 'The state became the site of the countrys first coronavirus death on Saturday.Updated 1m ago.More live coverage: .',
 'over the coronavirus, and researchers who

In [431]:
for sent in sents[:20]:
    print(process_text(sent),'\n\n')

['The workers conditions were not immediately available ,', 'public health officials in Alameda County and Solano County said in a news release that the workers were isolated in their homes . The news came after new cases were reported in both Washington State and Rhode Island .'] 


['Alex M. Azar II , the health and human services secretary , said on CBSs Face the Nation on Sunday that the government planned a radical expansion in testing capacity . Globally , the number of infections has risen to more than 88,000 . reported an additional 202 infections', '42 deaths from the virus , bringing its total number of confirmed infections to more than 80,000', 'total of deaths from the virus to exceed 3,000 .'] 


['[ Read : .'] 


['] .In Washington , two new patients in King County , both men in their 60s , were in critical condition , officials said .'] 


['The state became the site of the countrys first coronavirus death on Saturday . Updated 1 m ago . More live coverage : .'] 


['ove

In [156]:
any('subj' in mystr for mystr in ['subjpass', 'nsubj', 'onsubjpass'])

True

In [350]:
t = "Jammu and Kashmir has been put on alert to deal with any threat emerging due to coronavirus, even though no positive case has been found in the Union Territory so far, officials said."
doc = nlp(t)
noun_chunks = [chunk.text for chunk in doc.noun_chunks]
print(noun_chunks)
for tok in doc:
    if tok.dep_=='cc':
        left_right_tokens = [str(doc[tok.i-1]), str(doc[tok.i+1])]
        print(left_right_tokens)
        print([i for i in left_right_tokens if i in noun_chunks]=='')

['Jammu', 'Kashmir', 'alert', 'any threat', 'coronavirus', 'no positive case', 'the Union Territory', 'officials']
['Jammu', 'Kashmir']
False
